<a href="https://colab.research.google.com/github/metechmohit/Colab_Notebooks/blob/main/Josh_talks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets==2.16.0 fsspec==2023.9.2 accelerate torchcodec --quiet
!pip install pydub librosa soundfile --quiet
!pip install jiwer evaluate --quiet
!pip install pandas openpyxl --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 13.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2023.9.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import requests
import json
from pydub import AudioSegment
import io
import librosa
from datasets import Dataset, Audio
from tqdm import tqdm

import torch
import evaluate
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


## Data Processing and Chunking

In [3]:
df_urls = pd.read_csv("/content/FT Data - data.csv")

In [4]:
df_urls.head()

,user_id,recording_id,language,duration,rec_url_gcp,transcription_url_gcp,metadata_url_gcp
0,245746,825780,hi,443,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
1,291038,825727,hi,443,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
2,246004,988596,hi,475,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
3,93626,990175,hi,475,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
4,286851,526266,hi,522,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...


In [5]:
import os
if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.exists('data/audio_chunks'):
    os.makedirs('data/audio_chunks')

processed_data = []

# Initialize counters for rejected segments
rejected_long = 0
rejected_short = 0
rejected_redacted = 0
rejected_empty = 0

# Process each recording
for index, row in df_urls.iterrows():
    rec_id = row['recording_id']
    audio_url = row['rec_url_gcp']
    transcription_url = row['transcription_url_gcp']

    try:
        # Download transcription JSON
        transcription_response = requests.get(transcription_url)
        transcriptions = json.loads(transcription_response.text)

        # Download audio file
        audio_response = requests.get(audio_url)
        audio_data = io.BytesIO(audio_response.content)
        audio_segment = AudioSegment.from_wav(audio_data)

        # Process each segment from the transcription
        for segment in transcriptions:
            text = segment['text'].strip()

            # Get start and end times in milliseconds
            start_ms = int(segment['start'] * 1000)
            end_ms = int(segment['end'] * 1000)

            duration = end_ms - start_ms

            # Check filtering conditions and increment counters
            if duration > 30.0 * 1000:
                rejected_long += 1
                continue
            if duration < 1000:
                rejected_short += 1
                continue
            if text.upper() == "REDACTED":
                rejected_redacted += 1
                continue
            if not text:
                rejected_empty += 1
                continue

            # Chunk the audio and save it
            chunk = audio_segment[start_ms:end_ms]
            chunk_file_path = f'data/audio_chunks/{rec_id}_{start_ms}_{end_ms}.wav'
            chunk.export(chunk_file_path, format="wav")

            processed_data.append({
                'audio_path': chunk_file_path,
                'transcription': text
            })

        print(f"Processed recording {rec_id} successfully.")

    except Exception as e:
        print(f"Error processing recording {rec_id}: {e}")

# Save the processed data to a new CSV file
df_processed = pd.DataFrame(processed_data)
df_processed.to_csv('processed_data.csv', index=False)

# Display the rejection counts
print("\nSegment Rejection Summary:")
print(f"Rejected (longer than 30s): {rejected_long}")
print(f"Rejected (shorter than 1s): {rejected_short}")
print(f"Rejected (REDACTED text): {rejected_redacted}")
print(f"Rejected (empty text): {rejected_empty}")


# Display some information about the new dataset
print("\nProcessed Data Overview:")
print(df_processed.info())
print(df_processed.head())

Processed recording 825780 successfully.
Processed recording 825727 successfully.
Processed recording 988596 successfully.
Processed recording 990175 successfully.
Processed recording 526266 successfully.
Processed recording 520199 successfully.
Processed recording 542785 successfully.
Processed recording 494019 successfully.
Processed recording 523045 successfully.
Processed recording 522951 successfully.
Processed recording 254219 successfully.
Processed recording 253253 successfully.
Processed recording 351501 successfully.
Processed recording 350606 successfully.
Processed recording 629904 successfully.
Processed recording 635909 successfully.
Processed recording 989901 successfully.
Processed recording 990783 successfully.
Processed recording 240907 successfully.
Processed recording 240909 successfully.
Processed recording 270153 successfully.
Processed recording 270150 successfully.
Processed recording 475392 successfully.
Processed recording 475356 successfully.
Processed record

In [6]:
print(df_processed.shape)
df_processed.head(10)

(4741, 2)


,audio_path,transcription
0,data/audio_chunks/825780_110_14420.wav,अब काफी अच्छा होता है क्योंकि उनकी जनसंख्या बह...
1,data/audio_chunks/825780_14420_29030.wav,अनुभव करके कुछ लिखना था तो वह तो बिना देखिए नह...
2,data/audio_chunks/825780_29030_41840.wav,जंगल का सफर होता है जब हम रहने के लिए गए थे ना...
3,data/audio_chunks/825780_42470_50570.wav,पहली बारी था क्योंकि चलना नहीं आता न वहाँ का ज...
4,data/audio_chunks/825780_52700_66830.wav,हां तो फिर वहां जो दिन भर तो दिन में तो खोजने ...
5,data/audio_chunks/825780_66830_80510.wav,अजीब सा आवाज आने लगा बहुत अजीब सा डर तो इतना ल...
6,data/audio_chunks/825780_89060_103190.wav,डर लगता है लेकिन बहुत सारे थे तो पता सब अपना अ...
7,data/audio_chunks/825780_103190_117290.wav,ऐसा ही बात था तो पता है रात को मतलब जैसे छः सा...
8,data/audio_chunks/825780_117380_129919.wav,लगता है कि सर सुबह हो गया लेकिन कोई उठाने वाला...
9,data/audio_chunks/825780_130250_144830.wav,हम्म हम लोग की तो सुबह होती है की मम्मी उठाने ...


In [7]:
from datasets import Dataset, Audio
from sklearn.model_selection import train_test_split
import pandas as pd

df_processed = pd.read_csv('processed_data.csv')

# Create a Hugging Face Dataset from the DataFrame
dataset = Dataset.from_pandas(df_processed)

# This will automatically load and resample the audio to 16kHz
dataset = dataset.cast_column("audio_path", Audio(sampling_rate=16000))

# Rename the column to 'audio' as expected by the model
dataset = dataset.rename_column('audio_path', 'audio')

# Split the dataset into train and test sets
dataset_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

In [8]:
print("\nDataset split successfully.")
print(train_dataset)
print(eval_dataset)


Dataset split successfully.
Dataset({
    features: ['audio', 'transcription'],
    num_rows: 3792
})
Dataset({
    features: ['audio', 'transcription'],
    num_rows: 949
})


## Load Model, Processor, and Data Collator

In [9]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:

    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Split inputs and labels since they have to be of different lengths and need different padding methods
        # First, treat the audio inputs
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Second, treat the labels (transcriptions)
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Replace padding token with -100 to ignore it in loss calculation
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # If bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

# Load the Whisper processor (feature extractor + tokenizer)
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")

# Load the pre-trained model
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# Define a function to prepare the data for the model
def prepare_dataset(batch):
    # Load and resample audio on the fly (handled by datasets.Audio)
    audio = batch["audio"]

    # Process audio inputs to generate input_features
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # Process transcriptions to generate labels
    batch["labels"] = processor.tokenizer(batch["transcription"]).input_ids
    return batch

# Map the preparation function to the datasets
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(prepare_dataset, remove_columns=eval_dataset.column_names)

# Use the specialized data collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/3792 [00:00<?, ? examples/s]

Map:   0%|          | 0/949 [00:00<?, ? examples/s]

## Configure and Start Training

In [10]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Set up training arguments
from transformers import Seq2SeqTrainingArguments

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hindi-finetuned",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    num_train_epochs=3,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    report_to="tensorboard",
    fp16=True,
    push_to_hub=False,
    predict_with_generate=True,
)

# Instantiate the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
)

# Start training
print("Starting fine-tuning...")
trainer.train()
print("Fine-tuning complete.")

/tmp/ipython-input-3692025124.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Starting fine-tuning...


You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,0.498000,0.329575
1000,0.229700,0.312920


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4034: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Fine-tuning complete.


## Evaluation

In [11]:
import evaluate
from datasets import load_dataset
import numpy as np
import torch
import os
from transformers import WhisperForConditionalGeneration, WhisperProcessor

wer_metric = evaluate.load("wer")

print("Loading FLEURS Hindi test set...")
fleurs_hindi_test = load_dataset("google/fleurs", "hi_in", split="test")
print("FLEURS Hindi test set loaded successfully.")

Loading FLEURS Hindi test set...


/usr/local/lib/python3.12/dist-packages/datasets/load.py:1429: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

FLEURS Hindi test set loaded successfully.


In [12]:
from tqdm import tqdm
baseline_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to("cuda")
baseline_processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")

print("Evaluating baseline model...")
predictions = []
references = []

for item in tqdm(fleurs_hindi_test):
    audio = item['audio']
    input_features = baseline_processor.feature_extractor(audio['array'], sampling_rate=audio['sampling_rate']).input_features

    with torch.no_grad():
        predicted_ids = baseline_model.generate(torch.tensor(input_features).to("cuda"))

    transcription = baseline_processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    predictions.append(transcription)
    references.append(item['transcription'])

wer_baseline = wer_metric.compute(predictions=predictions, references=references)

print(f"Baseline Whisper-Small WER on FLEURS Hindi: {wer_baseline:.4f}")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating baseline model...


  0%|          | 0/418 [00:00<?, ?it/s]Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
100%|██████████| 418/418 [15:21<00:00,  2.20s/it]

Baseline Whisper-Small WER on FLEURS Hindi: 0.8464


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [14]:
# Step 2: Evaluate the Fine-Tuned Model
fine_tuned_model_path = os.path.join(training_args.output_dir, "/content/whisper-small-hindi-finetuned/checkpoint-1422")
fine_tuned_model = WhisperForConditionalGeneration.from_pretrained(fine_tuned_model_path).to("cuda")

print("\nEvaluating fine-tuned model...")
predictions = []
references = []

for item in tqdm(fleurs_hindi_test):
    audio = item['audio']
    input_features = baseline_processor.feature_extractor(audio['array'], sampling_rate=audio['sampling_rate']).input_features

    with torch.no_grad():
        predicted_ids = fine_tuned_model.generate(torch.tensor(input_features).to("cuda"))

    transcription = baseline_processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    predictions.append(transcription)
    references.append(item['transcription'])

wer_finetuned = wer_metric.compute(predictions=predictions, references=references)
print(f"Fine-Tuned Whisper-Small WER on FLEURS Hindi: {wer_finetuned:.4f}")


Evaluating fine-tuned model...


  0%|          | 0/418 [00:00<?, ?it/s]`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. If this is not desired, please set these values explicitly.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.l

Fine-Tuned Whisper-Small WER on FLEURS Hindi: 0.3925


In [15]:
# Final comparison
print(f"Baseline WER: {wer_baseline:.4f}")
print(f"Fine-tuned WER: {wer_finetuned:.4f}")
print(f"Absolute WER Reduction: {wer_baseline - wer_finetuned:.4f}")

Baseline WER: 0.8464
Fine-tuned WER: 0.3925
Absolute WER Reduction: 0.4539
